In [25]:
import os
import sys
import numpy as np
sys.path.append("/Users/atrask/Laboratory/clones/pycolab")
# from pycolab.engine import Engine
import pycolab.things as things
import pycolab.ascii_art as ascii_art
import pycolab.human_ui as human_ui
import curses
import torch
import six
import itertools
import collections

In [50]:
class RollingDrape(things.Drape):
  """A Drape that just `np.roll`s the mask around either axis."""

  # There are four rolls to choose from: two shifts of size 1 along both axes.
  _ROLL_AXES = [0, 0, 1, 1]
  _ROLL_SHIFTS = [-1, 1, -1, 1]

  def update(self, actions, board, layers, backdrop, all_things, the_plot):
    del board, layers, backdrop, all_things  # unused

    if actions is None: return  # No work needed to make the first observation.
    if actions == 4: the_plot.terminate_episode()  # Action 4 means "quit".

    # If the player has chosen a motion action, use that action to index into
    # the set of four rolls.
    if actions < 4:
      rolled = np.roll(self.curtain,  # Makes a copy, alas.
                       self._ROLL_SHIFTS[actions], self._ROLL_AXES[actions])
      np.copyto(self.curtain, rolled)
      the_plot.add_reward(1)  # Give ourselves a point for moving.

class SlidingSprite(things.Sprite):
  """A Sprite that moves in diagonal directions."""

  # We have four mappings from actions to motions to choose from. The mappings
  # are arranged so that given any index i, then across all sets, the motion
  # that undoes motion i always has the same index j.
  _DX = ([-1, 1, -1, 1], [-1, 1, -1, 1], [1, -1, 1, -1], [1, -1, 1, -1])
  _DY = ([-1, 1, 1, -1], [1, -1, -1, 1], [1, -1, -1, 1], [-1, 1, 1, -1])

  def __init__(self, corner, position, character, direction_set):
    """Build a SlidingSprite.
    Args:
      corner: required argument for Sprite.
      position: required argument for Sprite.
      character: required argument for Sprite.
      direction_set: an integer in `[0,3]` that selects from any of four
          mappings from actions to (diagonal) motions.
    """
    super(SlidingSprite, self).__init__(corner, position, character)
    self._dx = self._DX[direction_set]
    self._dy = self._DY[direction_set]

  def update(self, actions, board, layers, backdrop, all_things, the_plot):
    del board, layers, backdrop, all_things, the_plot  # unused
    # Actions 0-3 are motion actions; the others we ignore.
    if actions is None or actions > 3: return
    new_col = (self._position.col + self._dx[actions]) % self.corner.col
    new_row = (self._position.row + self._dy[actions]) % self.corner.row
    self._position = self.Position(new_row, new_col)

In [51]:
class Partial(object):
  """Holds a pycolab "thing" and its extra constructor arguments.
  In a spirit similar to `functools.partial`, a `Partial` object holds a
  subclass of one of the pycolab game entities described in `things.py`, along
  with any "extra" arguments required for its constructor (i.e. those besides
  the constructor arguments specified by the `things.py` base class
  constructors).
  `Partial` instances can be used to pass `Sprite`, `Drape` and `Backdrop`
  subclasses *and* their necessary "extra" constructor arguments to
  `ascii_art_to_game`.
  """

  def __init__(self, pycolab_thing, *args, **kwargs):
    """Construct a new Partial object.
    Args:
      pycolab_thing: a `Backdrop`, `Sprite`, or `Drape` subclass (note: not an
          object, the class itself).
      *args: "Extra" positional arguments for the `pycolab_thing` constructor.
      **kwargs: "Extra" keyword arguments for the `pycolab_thing` constructor.
    Raises:
      TypeError: `pycolab_thing` was not a `Backdrop`, a `Sprite`, or a `Drape`.
    """
    if not issubclass(pycolab_thing,
                      (things.Backdrop, things.Sprite, things.Drape)):
      raise TypeError('the pycolab_thing argument to ascii_art.Partial must be '
                      'a Backdrop, Sprite, or Drape subclass.')

    self.pycolab_thing = pycolab_thing
    self.args = args
    self.kwargs = kwargs

In [75]:
class Engine(object):
    
    def __init__(self, rows, cols, occlusion_in_layers=True):
        self._rows = rows
        self._cols = cols
        self._backdrop = None
        self._sprites_and_drapes = collections.OrderedDict()
        self._update_groups = collections.defaultdict(list)
        self._showtime = False

    def add_sprite(self, character, position, sprite_class, *args, **kwargs):
        self._runtime_error_if_called_during_showtime('add_sprite')
        self._value_error_if_characters_are_bad(character, mandatory_len=1)
        self._runtime_error_if_characters_claimed_already(character)
        if not issubclass(sprite_class, things.Sprite):
            raise TypeError('sprite_class arguments to Engine.add_sprite must be a '
                          'subclass of Sprite')
        if (not 0 <= position[0] < self._rows or
            not 0 <= position[1] < self._cols):
                raise ValueError('Position {} does not fall inside a {}x{} game board.'
                                 ''.format(position, self._rows, self._cols))

        # Construct the game board dimensions for the benefit of this sprite.
        corner = things.Sprite.Position(self._rows, self._cols)

        # Construct a new position for the sprite.
        position = things.Sprite.Position(*position)

        # Build and save the drape.
        sprite = sprite_class(corner, position, character, *args, **kwargs)
        self._sprites_and_drapes[character] = sprite
        self._update_groups[self._current_update_group].append(sprite)

        return sprite
    def set_agent(self, agent):
        self.agents = agent
        
    def reset(self):

        for thing in self.things:
            thing.create_world(self.rows, self.cols)
            
        self.agent.create_world(self.rows, self.cols)
        
    def render(self):
        dims = list()
        for cat, things in self.cat2things.items():
            dims.append(sum(map(lambda x:x.w,things)))
        dims.append(self.agent.w)
        return torch.cat(dims)

    def reward(self):
        return (sum(map(lambda x:x.render_reward_mask(),self.things)) * self.agent.w).sum()
        
    def update(self, y):
        left = torch.cat([self.agent.w[:,1:],self.agent.w[:,:1]], dim=1)
        right = torch.cat([self.agent.w[:,-1:],self.agent.w[:,:-1]], dim=1)
        up= torch.cat([self.agent.w[1:],self.agent.w[:1]], dim=0)
        down = torch.cat([self.agent.w[-1:],self.agent.w[:-1]], dim=0)
        stay = self.agent.w

        new_state = (y[0] * left) + (y[1] * right) + (y[2] * up) + (y[3] * down) + (y[4] * stay)
        self.agent.w = new_state

    def step(self, action):
        action = torch.FloatTensor([0,1,0,0,0])
        self.update(action)
        reward = self.reward()
        observation = self.render()
        return observation, reward
    
    def _runtime_error_if_called_during_showtime(self, method_name):
        if self._showtime:
            raise RuntimeError('{} should not be called after its_showtime() '
                               'has been called'.format(method_name))

    def _value_error_if_characters_are_bad(self, characters, mandatory_len=None):
        if mandatory_len is not None and len(characters) != mandatory_len:
            raise ValueError(
                '{}, a string of length {}, was used where a string of length {} was '
                'required'.format(repr(characters), len(characters), mandatory_len))
        for char in characters:
            try:               # This test won't catch all non-ASCII characters; if
                ord(char)        # someone uses a unicode string, it'll pass. But that's
            except TypeError:  # hard to do by accident.
                raise ValueError('Character {} is not an ASCII character'.format(char))            
    def _runtime_error_if_characters_claimed_already(self, characters):
        for char in characters:
            if self._backdrop and char in self._backdrop.palette:
                raise RuntimeError('Character {} is already being used by '
                                   'the backdrop'.format(repr(char)))
            if char in self._sprites_and_drapes:
                raise RuntimeError('Character {} is already being used by a sprite '
                                   'or a drape'.format(repr(char)))
            
    def update_group(self, group_name):
        """Change the update group for subsequent `add_sprite`/`add_drape` calls.
        The `Engine` consults `Sprite`s and `Drape`s for board updates in an order
        determined first by the update group name, then by the order in which the
        `Sprite` or `Drape` was added to the `Engine`. See the `Engine` constructor
        docstring for more details.
        It's fine to return to an update group after leaving it.
        Args:
          group_name: name of the new current update group.
        Raises:
          RuntimeError: if gameplay has already begun.
        """
        self._runtime_error_if_called_during_showtime('update_group')
        self._current_update_group = group_name
        
    def add_prefilled_drape(
        self, character, prefill, drape_class, *args, **kwargs):
        """Add a `Drape` to this `Engine`, with a custom initial mask.
        Much the same as `add_drape`, this method also allows callers to "prefill"
        the drape's `curtain` with an arbitrary mask. This method is mainly intended
        for use by the `ascii_art` tools; most `Drape` subclasses should fill their
        `curtain` on their own in the constructor (or in `update()`).
        Args:
        character: The ASCII character that this `Drape` directs the `Engine`
          to paint on the game board.
        prefill: 2-D `bool_` numpy array of the same dimensions as this `Engine`.
          The `Drape`'s curtain will be initialised with this pattern.
        drape_class: A subclass of `Drape` to be constructed by this method.
        *args: Additional positional arguments for the `drape_class` constructor.
        **kwargs: Additional keyword arguments for the `drape_class` constructor.
        Returns:
        the newly-created `Drape`.
        Raises:
        RuntimeError: if gameplay has already begun, or if any characters in
          `characters` has already been claimed by a preceding call to the
          `set_backdrop` or `add` methods.
        TypeError: if `drape_class` is not a `Drape` subclass.
        ValueError: if `character` is not a single ASCII character.
        """
        self._runtime_error_if_called_during_showtime('add_prefilled_drape')
        self._value_error_if_characters_are_bad(character, mandatory_len=1)
        self._runtime_error_if_characters_claimed_already(character)
        # Construct a new curtain for the drape.
        curtain = torch.ByteTensor(np.zeros((self._rows, self._cols), dtype=np.bool_))
        # Fill the curtain with the prefill data.
        curtain.set_(prefill)
        
        # Build and save the drape.
        drape = drape_class(curtain, character, *args, **kwargs)
        self._sprites_and_drapes[character] = drape
        self._update_groups[self._current_update_group].append(drape)
        
        return drape
    
    def set_z_order(self, z_order):
        """Set the z-ordering of all `Sprite`s and `Drape`s in this engine.
        Specify the complete order in which all `Sprite`s and `Drape`s should have
        their characters painted onto the game board. This method is available
        during game set-up only.
        Args:
          z_order: an ordered collection of all of the characters corresponding to
              all `Sprite`s and `Drape`s registered with this `Engine`.
        Raises:
          RuntimeError: if gameplay has already begun.
          ValueError: if the set of characters in `z_order` does not match the
              set of characters corresponding to all `Sprite`s and `Drape`s
              registered with this `Engine`.
        """
        self._runtime_error_if_called_during_showtime('set_z_order')
        if (set(z_order) != set(self._sprites_and_drapes.keys()) or
            len(z_order) != len(self._sprites_and_drapes)):
          raise ValueError('The z_order argument {} to Engine.set_z_order is not a '
                           'proper permutation of the characters corresponding to '
                           'Sprites and Drapes in this game, which are {}.'.format(
                               repr(z_order), self._sprites_and_drapes.keys()))
        new_sprites_and_drapes = collections.OrderedDict()
        for character in z_order:
          new_sprites_and_drapes[character] = self._sprites_and_drapes[character]
        self._sprites_and_drapes = new_sprites_and_drapes    

def ascii_art_to_long_tensor(art):
  """Construct a numpy array of dtype `uint8` from an ASCII art diagram.
  This function takes ASCII art diagrams (expressed as lists or tuples of
  equal-length strings) and derives 2-D numpy arrays with dtype `uint8`.
  Args:
    art: An ASCII art diagram; this should be a list or tuple whose values are
        all strings containing the same number of ASCII characters.
  Returns:
    A 2-D numpy array as described.
  Raises:
    ValueError: `art` wasn't an ASCII art diagram, as described; this could be
      because the strings it is made of contain non-ASCII characters, or do not
      have constant length.
    TypeError: `art` was not a list of strings.
  """
  error_text = (
      'the argument to ascii_art_to_uint8_nparray must be a list (or tuple) '
      'of strings containing the same number of strictly-ASCII characters.')
  try:
    art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)
  except ValueError as e:
    raise ValueError('{} (original error from numpy: {})'.format(error_text, e))
  except TypeError as e:
    if isinstance(art, (list, tuple)) and not all(
        isinstance(row, six.string_types) for row in art):
      error_text += ' Did you pass a list of list of single characters?'
    raise TypeError('{} (original error from numpy: {})'.format(error_text, e))
  if np.any(art > 127): raise ValueError(error_text)
  return torch.LongTensor(art)

def ascii_art_to_game(art,
                      what_lies_beneath,
                      sprites={},
                      drapes = {},
                      backdrop=things.Backdrop,
                      update_schedule=None,
                      z_order=None,
                      occlusion_in_layers=True):

    ### 1. Set default arguments, normalise arguments, derive various things ###
    
    # Partial() holds arbitrary arguments (args and kwargs) that you want to pass 
    #to CampX objects like sprites, drapes, and backdrops. 
    sprites = {char: sprite if isinstance(sprite, Partial) else Partial(sprite)
             for char, sprite in six.iteritems(sprites)}
    drapes = {char: drape if isinstance(drape, Partial) else Partial(drape)
            for char, drape in six.iteritems(drapes)}
    # Likewise, turn a bare Backdrop class into an argument-free Partial.
    if not isinstance(backdrop, Partial): backdrop = Partial(backdrop)

    # Compile characters corresponding to all Sprites and Drapes.
    non_backdrop_characters = set()
    non_backdrop_characters.update(sprites.keys())
    non_backdrop_characters.update(drapes.keys())
    if update_schedule is None: update_schedule = list(non_backdrop_characters)
        
    # If update_schedule is a string (someone wasn't reading the docs!),
    # gracefully convert it to a list of single-character strings.
    if isinstance(update_schedule, str): update_schedule = list(update_schedule)
        
    # If update_schedule is not a list-of-lists already, convert it to be one.
    if all(isinstance(item, str) for item in update_schedule):
        update_schedule = [update_schedule]
        
    ### 2. Argument checking and derivation of more... things ###
    
    # The update schedule (flattened) is the basis for the default z-order.
    try:
        flat_update_schedule = list(itertools.chain.from_iterable(update_schedule))
    except TypeError:
        raise TypeError('if any element in update_schedule is an iterable (like a '
                        'list), all elements in update_schedule must be')
    if set(flat_update_schedule) != non_backdrop_characters:
        raise ValueError('if specified, update_schedule must list each sprite and '
                         'drape exactly once.')

    # The default z-order is derived from there.
    if z_order is None: z_order = flat_update_schedule
    if set(z_order) != non_backdrop_characters:
        raise ValueError('if specified, z_order must list each sprite and drape '
                         'exactly once.')
        
    # All this checking is rather strict, but as this function is likely to be
  # popular with new users, it will help to fail with a helpful error message
    # now rather than an incomprehensible stack trace later.
    if isinstance(what_lies_beneath, str) and len(what_lies_beneath) != 1:
        raise ValueError(
            'what_lies_beneath may either be a single-character ASCII string or '
            'a list of ASCII-character strings')
        
    # Note that the what_lies_beneath check works for characters and lists both.
        try:
            _ = [ord(character) for character in ''.join(what_lies_beneath)]
            _ = [ord(character) for character in non_backdrop_characters]
            _ = [ord(character) for character in z_order]
            _ = [ord(character) for character in flat_update_schedule]
        except TypeError:
            raise ValueError(
                'keys of sprites, keys of drapes, what_lies_beneath (or its entries), '
                'values in z_order, and (possibly nested) values in update_schedule '
                'must all be single-character ASCII strings.')

    if non_backdrop_characters.intersection(''.join(what_lies_beneath)):
        raise ValueError(
            'any character specified in what_lies_beneath must not be one of the '
            'characters used as keys in the sprites or drapes arguments.')
        
    ### 3. Convert all ASCII art to numpy arrays ###
    
    art = ascii_art_to_long_tensor(HELLO_ART)
    
    # In preparation for masking out sprites and drapes from the ASCII art array
    # (to make the background), do similar for what_lies_beneath.
    if isinstance(what_lies_beneath, str):
        what_lies_beneath = torch.zeros_like(art) + ord(what_lies_beneath)
    else:
        what_lies_beneath = ascii_art_to_long_tensor(what_lies_beneath)
        if art.shape != what_lies_beneath.shape:
            raise ValueError(
                'if not a single ASCII character, what_lies_beneath must be ASCII '
                'art whose shape is the same as that of the ASCII art in art.')
            
    ### 4. Other miscellaneous preparation ###

    # This dict maps the characters associated with Sprites and Drapes to an
    # identifier for the update group to which they belong. The sorted order of
    # the identifiers matches the group ordering in update_schedule, but is
    # otherwise generic.
    update_group_for = {}
    for i, update_group in enumerate(update_schedule):
        group_id = '{:05d}'.format(i)
        update_group_for.update({character: group_id for character in update_group})

    ### 5. Construct engine; populate with Sprites and Drapes ###

    game = Engine(*art.shape, occlusion_in_layers=occlusion_in_layers)
    
    # Sprites and Drapes are added according to the depth-first traversal of the
    # update schedule.
    for character in flat_update_schedule:
        # Switch to this character's update group.
        game.update_group(update_group_for[character])
        # Find locations where this character appears in the ASCII art.
        mask = art == ord(character)

        if character in drapes:
            # Add the drape to the Engine.
            partial = drapes[character]
            game.add_prefilled_drape(character, mask,
                                     partial.pycolab_thing,
                                     *partial.args, **partial.kwargs)

        if character in sprites:
            # Get the location of the sprite in the ASCII art, if there was one.
            row, col = np.where(mask)
            if len(row) > 1:
                raise ValueError('sprite character {} can appear in at most one place '
                                 'in art.'.format(character))
            # If there was a location, convert it to integer values; otherwise, 0,0.
            # gpylint doesn't know how implicit bools work with numpy arrays...
            row, col = (int(row), int(col)) if len(row) > 0 else (0, 0)  # pylint: disable=g-explicit-length-test

            # Add the sprite to the Engine.
            partial = sprites[character]
            game.add_sprite(character, (row, col),
                          partial.pycolab_thing,
                          *partial.args, **partial.kwargs)

        # Clear out the newly-added Sprite or Drape from the ASCII art.
        art[mask] = what_lies_beneath[mask]

        ### 6. Impose specified Z-order ###
        game.set_z_order(z_order)


In [76]:
def make_game():
  """Builds and returns a Hello World game."""
  return ascii_art_to_game(
      HELLO_ART,
      what_lies_beneath=' ',
      sprites={'1': Partial(SlidingSprite, 0),
               '2': Partial(SlidingSprite, 1),
               '3': Partial(SlidingSprite, 2),
               '4': Partial(SlidingSprite, 3)},
      drapes={'@': RollingDrape},
      z_order='12@34')
make_game()

/Users/atrask/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:195: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


ValueError: The z_order argument '12@34' to Engine.set_z_order is not a proper permutation of the characters corresponding to Sprites and Drapes in this game, which are odict_keys(['@']).

In [146]:

HELLO_ART = ['                                    ',
             '  #   #  ### #    #     ###         ',
             '  #   # #    #    #    #   #        ',
             '  ##### #### #    #    #   #        ',
             '  #   # #    #    #    #   #        ',
             '  #   #  ###  ###  ###  ###         ',
             '                                    ',
             '     @   @  @@@   @@@  @    @@@@  1 ',
             '     @   @ @   @ @   @ @    @   @ 2 ',
             '     @ @ @ @   @ @@@@  @    @   @ 3 ',
             '     @ @ @ @   @ @   @ @    @   @   ',
             '      @@@   @@@  @   @  @@@ @@@@  4 ',
             '                                    ']

In [15]:
# Build a Hello World game.
game = make_game()

# Log a message in its Plot object.
game.the_plot.log('Hello, world!')

In [46]:
game.things

{'1': <__main__.SlidingSprite at 0x10d25d940>,
 '2': <__main__.SlidingSprite at 0x10d25dc50>,
 '3': <__main__.SlidingSprite at 0x10d25dcf8>,
 '4': <__main__.SlidingSprite at 0x10d25dc18>,
 '@': <__main__.RollingDrape at 0x10d25d898>}

In [18]:
board, reward, discount = game.its_showtime()

In [42]:
board, reward, discount = game.play(3)

In [43]:
discount

1.0

In [35]:
board

Observation(board=array([[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 32, 35, 32, 32, 35, 35, 35, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 32, 35, 35, 35, 32, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 32, 35, 32, 35, 32, 32, 32, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 35, 32, 32, 32, 35, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 35, 35, 35, 35, 32, 35, 35, 35, 35, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 35, 32, 32, 32, 35, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 32, 35, 32, 35, 32, 32, 32, 32, 35, 32, 32,
        32, 32, 35, 32, 32, 32, 32, 35, 32, 32, 32, 35, 32, 32, 32, 32,
        32, 32, 32, 32],
       [32, 32, 35, 32, 32, 64, 35, 32, 32, 64, 35, 35, 64, 64, 64, 35,
        35, 32, 64, 64, 64, 35, 32, 64, 35, 35, 35, 32, 64, 64, 6

In [9]:



# Let the game begin!
ui.play(game)

error: curs_set() returned ERR

In [50]:
engine = Engine(rows=3, cols=3)
backdrop = engine.set_backdrop('.asdfkjaa', Backdrop)
sprite = engine.add_sprite('C', (0, 0), SlidingSprite, 0)

In [30]:
backdrop.curtain

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [31]:
backdrop.curtain

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [1]:
import torch

from campx.thing import Thing
from campx.agent import Agent
from campx.world import TensorWorld

In [2]:
world = TensorWorld(3,3, agent=Agent(1,1))
world.add_thing(Thing(0,0, hover_reward=1, category="apple")) # good thing
world.add_thing(Thing(1,2, hover_reward=-1, category="snake")) # bad thing
world.add_thing(Thing(1,0, hover_reward=-1, category="snake")) # bad thing

In [3]:
world.reset()

In [4]:
observation = world.render()
observation


    1     0     0
    0     0     0
    0     0     0
    0     0     0
    1     0     1
    0     0     0
    0     0     0
    0     1     0
    0     0     0
[torch.FloatTensor of size 9x3]

In [5]:
action = torch.FloatTensor([0,1,0,0,0])
observation, reward = world.step(action)

In [6]:
reward

-1.0

In [7]:
observation


    1     0     0
    0     0     0
    0     0     0
    0     0     0
    1     0     1
    0     0     0
    0     0     0
    0     0     1
    0     0     0
[torch.FloatTensor of size 9x3]